# 물류 네트워크 생산 계획 최적화

In [3]:
import pandas as pd
import numpy as np

In [4]:
#데이터 불러오기
df_material = pd.read_csv('product_plan_material.csv', index_col='제품')
df_profit = pd.read_csv('product_plan_profit.csv', index_col='제품')
df_stock = pd.read_csv('product_plan_stock.csv', index_col='항목')
df_plan = pd.read_csv('product_plan.csv', index_col='제품')

In [5]:
df_material

,원료1,원료2,원료3
제품,,,
제품1,1,4,3
제품2,2,4,1


In [6]:
df_profit

,이익
제품,
제품1,5.0
제품2,4.0


In [7]:
df_stock

,원료1,원료2,원료3
항목,,,
재고,40,80,50


In [8]:
df_plan

,생산량
제품,
제품1,16
제품2,0


## 현재 총 이익 계산

In [22]:
#이익 계산 함수

def product_plan(df_profit, df_plan):

    profit = 0

    for i in range(len(df_profit)):
        for j in range(len(df_plan.columns)):
            profit += df_profit.iloc[i][j] * df_plan.iloc[i][j]

    return profit

In [24]:
print("총 이익:", product_plan(df_profit, df_plan))

총 이익: 80.0


## 생산 최적화 

In [25]:
from pulp import LpVariable, lpSum, value
from ortoolpy import model_max, addvars, addvals

In [26]:
df = df_material.copy()
df

,원료1,원료2,원료3
제품,,,
제품1,1,4,3
제품2,2,4,1


In [27]:
inv = df_stock
inv

,원료1,원료2,원료3
항목,,,
재고,40,80,50


In [35]:
df_profit

,이익
제품,
제품1,5.0
제품2,4.0


In [61]:
m = model_max()  # 최대하 모델 선언

#제품 변수 생산
v1 = {(i): LpVariable('v%d'%(i), lowBound=0) for i in range(len(df_profit))}
# 최대화 할 목적 함수 생산
m += lpSum(df_profit.iloc[i]* v1[i] for i in range(len(df_profit)))
#제약 조건 설정
for i in range(len(df.columns)):
    m += lpSum(df.iloc[j,i] * v1[j] for j in range(len(df_profit))) <= inv.iloc[:,i]

m.solve()


1

In [62]:
df_plan_sol = df_plan.copy()
for k , x in v1.items():
    df_plan_sol.iloc[k] = value(x)

print(df_plan_sol)
print('총 이익:' , value(m.objective))

      생산량
제품       
제품1    15
제품2     5
총 이익: 95.0


In [64]:
df_plan

,생산량
제품,
제품1,16
제품2,0


 - 제품1의  생산량을 1 줄이고 제품 2의 생산량을 5개 늘리면 이익을 95만원까지 기존 보다 15만원 향상할 수 있음

## 제약 조건 확인

In [65]:
df_material

,원료1,원료2,원료3
제품,,,
제품1,1,4,3
제품2,2,4,1


In [67]:
df_plan

,생산량
제품,
제품1,16
제품2,0


In [68]:
df_stock

,원료1,원료2,원료3
항목,,,
재고,40,80,50


In [77]:
def condition_stock(df_plan, df_material, df_stock):
    flag = np.zeros(len(df_material.columns))
    for i in range(len(df_material.columns)):
        temp_sum = 0
        for j in range(len(df_material)):
            temp_sum += (df_material.iloc[j][i] * float(df_plan.iloc[j]))
        
        if (temp_sum <=float(df_stock.iloc[0][i])):
            flag[i]=1
        print(df_material.columns[i]+ " 사용량:" + str(temp_sum) + " , 재고:" + str(float(df_stock.iloc[0][i])))
        
    return flag


print('최적화 전 계산 결과:' + str(condition_stock(df_plan, df_material, df_stock)))
print('----------최적화 후-------------------')
print('제약 조건 계산 결과:' + str(condition_stock(df_plan_sol, df_material, df_stock)))



원료1 사용량:16.0 , 재고:40.0
원료2 사용량:64.0 , 재고:80.0
원료3 사용량:48.0 , 재고:50.0
최적화 전 계산 결과:[1. 1. 1.]
----------최적화 후-------------------
원료1 사용량:25.0 , 재고:40.0
원료2 사용량:80.0 , 재고:80.0
원료3 사용량:50.0 , 재고:50.0
제약 조건 계산 결과:[1. 1. 1.]


- 최적화 이후 원료 2와 원료 3은 모두 사용 했으며, 원료 1이 조금 남아 있지만 최적화 계산 전과 비교하면 원료 사용 효율이 크게 개선 된 것을 알 수 있음